#### Autotrain 을 가지고 Llama2 데이터 학습 : PEFT
    * Autotrain 을 로컬데이터를 학습할 수 있는 방법을 찾지 못함.
    * 허깅페이스의 자원을 이용해서 모델학습을 시켜야 함.
```
1. excute: https://hackernoon.com/ko/%EC%A7%91%EC%97%90%EC%84%9C-%EB%A7%8C%EB%93%A0-%EB%8C%80%ED%98%95-%EC%96%B8%EC%96%B4-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EC%8B%A4%ED%97%98%ED%95%98%EB%8A%94-%EC%95%8C%ED%8C%8C%EC%B9%B4-%EB%A1%9C%EB%9D%BC
2. model: Llama2 pytouch.bin
3. parameter data type: touch.float16
```

#### requiment modules
```
pip install autotrain-advanced
pip install transformers
pip install bitsandbytes
pip install accelerate
pip install sentencepiece
pip install protobuf
pip install xformers
```

In [ ]:
# autotrain 라이브러리 설치
%pip install autotrain-advanced -q


In [ ]:
# autotrain 라이브러리 업데이트
!autotrain setup --update-torch

In [ ]:
# 학습을 위한 환경 변수 설정
import os
os.environ["PROJECT_NAME"] = "kocasalpha-sllm7b"
os.environ["MODEL_NAME"] = "/data/aibiseo/models/casllm-base-7b-hf"
os.environ["DATA_PATH"] = "/data/aibiseo/dataset/alpaca_en_dataset.jsonl"
os.environ["TEXT_COLUMN"] = "text"
os.environ["QUANTIZATION"] = "int4"



In [ ]:
import os
import pandas as pd
import json
import jsonlines

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk("/data/aibiseo/dataset/alpaca_en_dataset.jsonl")


In [ ]:
dataset[:5]

In [ ]:
# Works on A100 80G x4
!torchrun --nproc_per_node=4 --master_port=34321 ../script/train/run_clm.py \
--model_name_or_path='/data/aibiseo/models/casllm-base-7b-hf' \
--train_file='/data/aibiseo/dataset/en_alpaca_data.json' \
--num_train_epochs=2 \
--block_size=1024 \
--per_device_train_batch_size=1 \
--gradient_accumulation_steps=64 \
--torch_dtype=float16 \
--fp16 \
--output_dir='/data/aibiseo/models/kocasalpha-sllm7b' \
--deepspeed=ds_zero3-nooffload.json \
--do_train \
--save_strategy='epoch' \
--logging_strategy='steps' \
--logging_first_step \
--save_total_limit=1 \
--run_name='polyglot-12.8b-koalpaca-v1.1b-ga64'

In [ ]:
!autotrain llm --train \
    --project-name "/data/aibiseo/models/kocasalpha-sllm7b" \
    --model "/data/aibiseo/models/casllm-base-7b-hf" \
    --data-path "/data/aibiseo/dataset/alpaca_en_dataset.jsonl" \
    --text_column "dataset" \
    --peft\
    --quantization "int4"\
    --lr 2e-4 \
    --auto-find-batch-size \
    --epochs 1 \
    --trainer sft \
    --block_size 4096 \
    --model_max_length 4096